In [ ]:
from tensorscaling import scale, unit_tensor, random_tensor, marginal
import numpy as np

# Tensor scaling

Scale 3x3x3 unit tensor to certain non-uniform marginals:

In [ ]:
shape = [3, 3, 3]
targets = [(.5, .25, .25), (.4, .3, .3), (.7, .2, .1)]

res = scale(unit_tensor(3, 3), targets, eps=1e-4)
res

Result(success=True, iterations=88, max_dist=9.620242008657933e-05, ..., log_cap=-0.383593921056698)

We can also access the scaling matrices and the final scaled state:

In [ ]:
print(res.gs[0], "\n")
print(res.gs[1], "\n")
print(res.gs[2])

[[ 1.88670152+0.j          0.        +0.j          0.        +0.j        ]
 [-0.14495687-0.59508447j  0.62903469+0.j          0.        +0.j        ]
 [-0.20094453+0.4156131j   0.00995373-0.22155697j  0.61764174+0.j        ]] 

[[ 1.36219588+0.j          0.        +0.j          0.        +0.j        ]
 [ 0.57570639-0.77762158j  0.53376874+0.j          0.        +0.j        ]
 [-0.27727244-0.09549325j -1.23051413-1.10242138j  1.83328947+0.j        ]] 

[[ 1.77541047+0.j          0.        +0.j          0.        +0.j        ]
 [ 0.20001441-0.38552597j  0.53991354+0.j          0.        +0.j        ]
 [ 0.07271761-0.13632339j -0.01869868-0.15336731j  0.39394219+0.j        ]]


Let's now check that the W tensor *cannot* be scaled to uniform marginals:

In [ ]:
shape = [2, 2, 2, 2]
W = np.zeros(shape)
W[1, 0, 0, 0] = W[0, 1, 0, 0] = W[0, 0, 1, 0] = W[0, 0, 0, 1] = .5
targets = [(.5, .5)] * 4

scale(W, targets, eps=1e-4, max_iterations=1000)

Result(success=False, iterations=1000, max_dist=0.20301485712543133, ..., log_cap=None)

To see more clearly what is going on, we can set the `verbose` flag:

In [ ]:
res = scale(W, targets, eps=1e-4, max_iterations=10, verbose=True)

scaling tensor of shape (2, 2, 2, 2) and type float64
target spectra:
  0: (0.5, 0.5)
  1: (0.5, 0.5)
  2: (0.5, 0.5)
  3: (0.5, 0.5)
#000: max_dist = 0.35355339 @ sys = 1
#001: max_dist = 0.47140452 @ sys = 2
#002: max_dist = 0.56568542 @ sys = 3
#003: max_dist = 0.62853936 @ sys = 0
#004: max_dist = 0.58232323 @ sys = 1
#005: max_dist = 0.59305730 @ sys = 2
#006: max_dist = 0.59545834 @ sys = 3
#007: max_dist = 0.59262283 @ sys = 0
#008: max_dist = 0.59353004 @ sys = 1
#009: max_dist = 0.59357133 @ sys = 2
#010: max_dist = 0.59340661 @ sys = 3
did not converge!


We see that at each point in the algorithm, one of the marginals has Frobenius distance $\approx 0.59$ to being uniform. Indeed, we know that the entanglement polytope of the W tensor does not include the point corresponding to uniform marginals -- see [here](https://www.entanglement-polytopes.org/four_qubits) for an interactive visualization!

# Tuples of matrices and tensors

We can just as well only prescribe the desired spectra for subsystems.
Note that prescribing two out of three marginals amounts to *operator scaling*.

In [ ]:
shape = [3, 3, 3]
targets = [(.4, .3, .3), (.7, .2, .1)]

res = scale(unit_tensor(3, 3), targets, eps=1e-6)
res

Result(success=True, iterations=31, max_dist=7.783747239854942e-07, ..., log_cap=-0.24662956299578093)

Indeed, the last two marginals are as prescribed, while the first marginal is arbitrary.

In [ ]:
print(marginal(res.psi, 0).round(5), "\n")
print(marginal(res.psi, 1).round(5), "\n")
print(marginal(res.psi, 2).round(5))

[[0.30884-0.j      0.00585-0.0084j  0.01166+0.00214j]
 [0.00585+0.0084j  0.35411+0.j      0.00326+0.02375j]
 [0.01166-0.00214j 0.00326-0.02375j 0.33705+0.j     ]] 

[[ 0.4+0.j -0. +0.j  0. +0.j]
 [-0. -0.j  0.3-0.j  0. +0.j]
 [ 0. -0.j  0. -0.j  0.3+0.j]] 

[[ 0.7-0.j  0. -0.j -0. -0.j]
 [ 0. +0.j  0.2-0.j -0. -0.j]
 [-0. +0.j -0. -0.j  0.1+0.j]]
